<a href="https://colab.research.google.com/github/halimahbatam18/Tugas-AI/blob/main/pertemuan_10_sentiment_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Needed Package

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
!is /content/drive//MyDrive/

/bin/bash: line 1: is: command not found


In [ ]:
# ===== Step 1: Import library & cek data =====
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Jika file belum dibaca sebelumnya, uncomment dan isi path yang benar:
# data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

# Tampilkan info dasar untuk memastikan data terbaca
print("Pandas:", pd.__version__)
print("TensorFlow:", tf.__version__)
try:
    print("Rows:", len(data))
    print(data['sentiment'].value_counts())
    display(data.head())
except NameError:
    print("Variabel 'data' belum ada. Pastikan kamu sudah menjalankan cell yang memuat pd.read_csv(...)")

Pandas: 2.2.2
TensorFlow: 2.19.0
Rows: 50000
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
# ===== Cell 3: Split data & Tokenizing =====

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1) Pisahkan teks & label
X = data['clean_review'].values
y = data['sentiment'].values

# 2) Split train dan test (80% : 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data latih:", len(X_train))
print("Data uji:", len(X_test))

# 3) Tokenisasi (ubah kata jadi angka)
vocab_size = 20000
max_len = 150
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

# ubah teks ke deretan angka
seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)

# 4) Padding supaya panjangnya sama
X_train_pad = pad_sequences(seq_train, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(seq_test, maxlen=max_len, padding='post', truncating='post')

print("Bentuk data latih:", X_train_pad.shape)
print("Bentuk data uji:", X_test_pad.shape)

Data latih: 40000
Data uji: 10000
Bentuk data latih: (40000, 150)
Bentuk data uji: (10000, 150)


In [ ]:
# ===== Debug + Run Cell untuk Split & Tokenizing =====
import sys, traceback
print("Python:", sys.version.splitlines()[0])
try:
    import pandas as pd
    import numpy as np
    import re
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import tensorflow as tf
    print("pandas", pd.__version__, "tensorflow", tf.__version__)
except Exception as e:
    print("Error saat import library:")
    traceback.print_exc()

# 1) cek apakah 'data' ada dan kolomnya benar
try:
    print("\nVar 'data' exists:", 'data' in globals() or 'data' in locals())
    print("Tipe data:", type(data))
    print("Kolom:", data.columns.tolist())
    print("Contoh baris (review, sentiment):")
    display(data[['review','sentiment']].head(3))
except Exception:
    print("Gagal mengakses 'data' atau kolom tidak ditemukan:")
    traceback.print_exc()

# 2) Jalankan split + tokenizing di try/except untuk melihat error lengkap
try:
    X = data['clean_review'].values
    y = data['sentiment'].values
    print("\nContoh clean_review:", X[:2])
    print("Contoh label:", y[:6])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("\nData latih:", len(X_train), "Data uji:", len(X_test))

    # tokenisasi
    vocab_size = 20000
    max_len = 150
    oov_tok = "<OOV>"
    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
    tokenizer.fit_on_texts(X_train)

    seq_train = tokenizer.texts_to_sequences(X_train)
    seq_test = tokenizer.texts_to_sequences(X_test)

    X_train_pad = pad_sequences(seq_train, maxlen=max_len, padding='post', truncating='post')
    X_test_pad = pad_sequences(seq_test, maxlen=max_len, padding='post', truncating='post')

    print("\nBentuk data latih:", X_train_pad.shape)
    print("Bentuk data uji:", X_test_pad.shape)
except Exception:
    print("\nTerjadi error saat split/tokenizing:")
    traceback.print_exc()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
pandas 2.2.2 tensorflow 2.19.0

Var 'data' exists: True
Tipe data: <class 'pandas.core.frame.DataFrame'>
Kolom: ['review', 'sentiment']
Contoh baris (review, sentiment):


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1



Terjadi error saat split/tokenizing:


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'clean_review'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipython-input-3258021340.py", line 30, in <cell line: 0>
    X = data['clean_review'].values
        ~~~~^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/core/frame.py", line 4102, in __getitem__
    indexer = self.columns.get_loc(key)
  

In [ ]:
display(data.tail())

import pandas as pd

# Baca dataset
data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

# Tampilkan 5 data terakhir
data.tail()

# Hitung jumlah label positif dan negatif
print(data["sentiment"].value_counts())

# Ganti label ke angka
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

# Cek hasil konversi
data.head()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


sentiment
positive    25000
negative    25000
Name: count, dtype: int64


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

X_train, X_test

(array([[1935,    1, 1200, ...,  205,  351, 3856],
        [   3, 1651,  595, ...,   89,  103,    9],
        [   0,    0,    0, ...,    2,  710,   62],
        ...,
        [   0,    0,    0, ..., 1641,    2,  603],
        [   0,    0,    0, ...,  245,  103,  125],
        [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32),
 array([[   0,    0,    0, ...,  995,  719,  155],
        [  12,  162,   59, ...,  380,    7,    7],
        [   0,    0,    0, ...,   50, 1088,   96],
        ...,
        [   0,    0,    0, ...,  125,  200, 3241],
        [   0,    0,    0, ..., 1066,    1, 2305],
        [   0,    0,    0, ...,    1,  332,   27]], dtype=int32))

In [ ]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

y_train, y_test

(39087    0
 30893    0
 45278    1
 16398    0
 13653    0
         ..
 11284    1
 44732    1
 38158    0
 860      1
 15795    1
 Name: sentiment, Length: 40000, dtype: int64,
 33553    1
 9427     1
 199      0
 12447    1
 39489    0
         ..
 28567    0
 25079    1
 18707    1
 15200    0
 5857     1
 Name: sentiment, Length: 10000, dtype: int64)

In [ ]:
# MODEL TRAINING
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.summary()

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 212s 417ms/step - accuracy: 0.7273 - loss: 0.5252 - val_accuracy: 0.8413 - val_loss: 0.3744
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 420ms/step - accuracy: 0.8543 - loss: 0.3540 - val_accuracy: 0.8621 - val_loss: 0.3306
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 426ms/step - accuracy: 0.8694 - loss: 0.3201 - val_accuracy: 0.8587 - val_loss: 0.3568
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 210s 421ms/step - accuracy: 0.8867 - loss: 0.2842 - val_accuracy: 0.8586 - val_loss: 0.3356
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 418ms/step - accuracy: 0.9050 - loss: 0.2431 - val_accuracy: 0.8626 - val_loss: 0.3531


In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Akurasi data test:", acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 118ms/step - accuracy: 0.8714 - loss: 0.3380
Akurasi data test: 0.8734999895095825
